In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

from matplotlib import pyplot as plt
from scipy.stats import boxcox

from google.colab import files

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
season1 = pd.read_csv('https://raw.githubusercontent.com/rzmcinty/HONproject/main/scraperdata2122.csv') # 2021-2022 regseason
#print(season1)
season2 = pd.read_csv('https://raw.githubusercontent.com/rzmcinty/HONproject/main/scraperdata-2223.csv') # 2022-2023  reg season
#print(season2.shape)
season3 = pd.read_csv('https://raw.githubusercontent.com/rzmcinty/HONproject/main/scraperdata-2324.csv') # 2023-2024 reg season
#season3.shape

In [ ]:
url = 'https://www.hockey-reference.com/leagues/NHL_2022_games.html'
dfs = pd.read_html(url)
#print(dfs)
reg1dates = dfs[0]
reg1dates

reg1dates.replace(to_replace="St. Louis Blues", value="St Louis Blues")
reg1dates = reg1dates.replace(to_replace="St. Louis Blues", value="St Louis Blues")
reg1dates

,Date,Visitor,G,Home,G.1,Unnamed: 5,Att.,LOG,Notes
0,2021-10-12,Pittsburgh Penguins,6,Tampa Bay Lightning,2,NaN,19092.0,2:33,NaN
1,2021-10-12,Seattle Kraken,3,Vegas Golden Knights,4,NaN,18431.0,2:25,NaN
2,2021-10-13,Winnipeg Jets,1,Anaheim Ducks,4,NaN,16260.0,2:29,NaN
3,2021-10-13,Chicago Blackhawks,2,Colorado Avalanche,4,NaN,18037.0,2:32,NaN
4,2021-10-13,Vancouver Canucks,2,Edmonton Oilers,3,SO,16034.0,2:42,NaN
...,...,...,...,...,...,...,...,...,...
1307,2022-04-29,San Jose Sharks,0,Seattle Kraken,3,NaN,17151.0,2:16,NaN
1308,2022-04-29,Vegas Golden Knights,7,St Louis Blues,4,NaN,18096.0,2:23,NaN
1309,2022-04-29,Boston Bruins,2,Toronto Maple Leafs,5,NaN,18219.0,2:29,NaN
1310,2022-04-29,Calgary Flames,1,Winnipeg Jets,3,NaN,14202.0,2:21,NaN


In [ ]:
url = 'https://www.hockey-reference.com/leagues/NHL_2023_games.html'
dfs = pd.read_html(url)
#print(dfs)
reg2dates = dfs[0]
print(type(reg2dates))
reg2dates = reg2dates.replace(to_replace="St. Louis Blues", value="St Louis Blues")
reg2dates

<class 'pandas.core.frame.DataFrame'>


,Date,Visitor,G,Home,G.1,Unnamed: 5,Att.,LOG,Notes
0,2022-10-07,San Jose Sharks,1,Nashville Predators,4,NaN,16648,2:43,"at O2 Arena (Prague, CZ)"
1,2022-10-08,Nashville Predators,3,San Jose Sharks,2,NaN,17023,2:33,"at O2 Arena (Prague, CZ)"
2,2022-10-11,Vegas Golden Knights,4,Los Angeles Kings,3,NaN,18230,2:31,NaN
3,2022-10-11,Tampa Bay Lightning,1,New York Rangers,3,NaN,18006,2:21,NaN
4,2022-10-12,Seattle Kraken,4,Anaheim Ducks,5,OT,17530,2:28,NaN
...,...,...,...,...,...,...,...,...,...
1307,2023-04-13,Vegas Golden Knights,3,Seattle Kraken,1,NaN,17151,2:32,NaN
1308,2023-04-13,Detroit Red Wings,0,Tampa Bay Lightning,5,NaN,19092,2:22,NaN
1309,2023-04-13,New Jersey Devils,5,Washington Capitals,4,OT,18573,2:33,NaN
1310,2023-04-14,Buffalo Sabres,5,Columbus Blue Jackets,2,NaN,18786,2:15,NaN


In [ ]:
url = 'https://www.hockey-reference.com/leagues/NHL_2024_games.html'
dfs = pd.read_html(url)
#print(dfs)
reg3dates = dfs[0]
print(type(reg3dates))
#reg3dates
reg3dates = reg3dates.replace(to_replace="St. Louis Blues", value="St Louis Blues")

<class 'pandas.core.frame.DataFrame'>


In [ ]:
nhl_teams = [
    "Anaheim Ducks",
    "Arizona Coyotes",
    "Boston Bruins",
    "Buffalo Sabres",
    "Calgary Flames",
    "Carolina Hurricanes",
    "Chicago Blackhawks",
    "Colorado Avalanche",
    "Columbus Blue Jackets",
    "Dallas Stars",
    "Detroit Red Wings",
    "Edmonton Oilers",
    "Florida Panthers",
    "Los Angeles Kings",
    "Minnesota Wild",
    "Montreal Canadiens",
    "Nashville Predators",
    "New Jersey Devils",
    "New York Islanders",
    "New York Rangers",
    "Ottawa Senators",
    "Philadelphia Flyers",
    "Pittsburgh Penguins",
    "San Jose Sharks",
    "Seattle Kraken",  # Newly added team
    "St Louis Blues",
    "Tampa Bay Lightning",
    "Toronto Maple Leafs",
    "Vancouver Canucks",
    "Vegas Golden Knights",
    "Washington Capitals",
    "Winnipeg Jets",
]
nhl_teams_abbr = [
    "ANA",  # Anaheim Ducks
    "ARI",  # Arizona Coyotes
    "BOS",  # Boston Bruins
    "BUF",  # Buffalo Sabres
    "CGY",  # Calgary Flames
    "CAR",  # Carolina Hurricanes
    "CHI",  # Chicago Blackhawks
    "COL",  # Colorado Avalanche
    "CBJ",  # Columbus Blue Jackets
    "DAL",  # Dallas Stars
    "DET",  # Detroit Red Wings
    "EDM",  # Edmonton Oilers
    "FLA",  # Florida Panthers
    "LAK",  # Los Angeles Kings
    "MIN",  # Minnesota Wild
    "MTL",  # Montreal Canadiens
    "NSH",  # Nashville Predators
    "NJD",  # New Jersey Devils
    "NYI",  # New York Islanders
    "NYR",  # New York Rangers
    "OTT",  # Ottawa Senators
    "PHI",  # Philadelphia Flyers
    "PIT",  # Pittsburgh Penguins
    "SJS",  # San Jose Sharks
    "SEA",  # Seattle Kraken # Newly added team
    "STL",  # St Louis Blues
    "TBL",  # Tampa Bay Lightning
    "TOR",  # Toronto Maple Leafs
    "VAN",  # Vancouver Canucks
    "VGK",  # Vegas Golden Knights
    "WSH",  # Washington Capitals
    "WPG",  # Winnipeg Jets
]


In [ ]:
# Replace 'TEAM_NAME' with the name of the team you want to filter
team_name = nhl_teams[25]

# Filter the DataFrame to get all games played by the specified team
team_games_df = season1[season1['Team'] == team_name]

# Verify the length of the DataFrame to ensure it contains 82 games
if len(team_games_df) == 82:
    print("All 82 games found for", team_name)
else:
    print("The number of games found for", team_name, "is not 82")

# If you want to reset the index of the resulting DataFrame
team_games_df.reset_index(drop=True, inplace=True)


print(team_games_df.iloc[[1],[0]])

All 82 games found for St Louis Blues
             Team
1  St Louis Blues


In [ ]:
team_games_df = season2[season2['Team'] == "St Louis Blues"] #shout out data science guru at hunt library issues with St. Louis vs St Louis
team_games_df

,Team,GP,TOI/GP,W,L,OTL,ROW,Points,Point %,CF/60,...,LDSF%,LDGF/60,LDGA/60,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Date
65,St Louis Blues,1,53:04,1,0,0,1,2,1.0,49.75,...,55.56,0.00,0.00,-,0.0,100.0,19.05,91.67,1.107,2022-10-15
114,St Louis Blues,1,54:10,1,0,0,1,2,1.0,35.45,...,33.33,1.11,1.11,50.0,14.29,92.86,16.67,91.43,1.081,2022-10-19
152,St Louis Blues,1,56:59,1,0,0,1,2,1.0,40.01,...,38.89,0.00,0.00,-,0.0,100.0,5.26,100.00,1.053,2022-10-22
188,St Louis Blues,1,47:03,0,1,0,0,0,0.0,62.49,...,50.00,0.00,1.28,0.00,0.0,90.0,0.00,88.89,0.889,2022-10-24
217,St Louis Blues,1,50:16,0,1,0,0,0,0.0,54.91,...,64.71,0.00,0.00,-,0.0,100.0,0.00,86.96,0.870,2022-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,St Louis Blues,1,48:00,1,0,0,1,2,1.0,63.75,...,50.00,1.25,0.00,100.00,7.14,100.0,14.29,93.33,1.076,2023-04-04
2497,St Louis Blues,1,58:01,1,0,0,1,2,1.0,44.47,...,42.86,1.03,0.00,100.00,16.67,100.0,14.29,100.00,1.143,2023-04-06
2510,St Louis Blues,1,47:47,0,1,0,0,0,0.0,75.34,...,57.89,0.00,0.00,-,0.0,100.0,8.33,90.48,0.988,2023-04-08
2587,St Louis Blues,1,49:12,0,1,0,0,0,0.0,35.37,...,33.33,0.00,1.22,0.0,0.0,90.0,7.14,91.67,0.988,2023-04-12


In [ ]:
season1_dict ={}

for i in range(len(nhl_teams)):
  # Replace 'TEAM_NAME' with the name of the team you want to filter
  team_name = nhl_teams[i]

  # Filter the DataFrame to get all games played by the specified team
  team_games_df = season1[season1['Team'] == team_name]

  # Verify the length of the DataFrame to ensure it contains 82 games
  if len(team_games_df) == 82:
      print("All 82 games found for", team_name)
  else:
      print("The number of games found for", team_name, "is not 82:",len(team_games_df),"Games found.: i =", i)


  # If you want to reset the index of the resulting DataFrame
  team_games_df.reset_index(drop=True, inplace=True)


  team_name = nhl_teams[i] #i
  team_games_df = season1[season1['Team'] == team_name]
  # Loop through each game from game 10 to 82 (inclusive)
  # Create an empty list to store dictionaries
  moving_averages_list = []
  df = team_games_df
  df_copy = df.copy()
  df_copy.replace('-', np.nan, inplace=True)
  exclude_columns = ['TOI/GP', 'Team', 'Date']


  for game_index in range(10, 82):
      # Select data from game 1 to the current game, excluding non-numeric columns
      subset_df = df_copy.drop(columns=exclude_columns).iloc[:game_index]

      # Calculate the moving averages for all statistics
      moving_averages = subset_df.rolling(window=10, min_periods=10).mean().iloc[-1]

      # Get the date of the next game
      next_game_index = game_index   # Index of the next game
      #print("Next game index:", next_game_index)

      next_game_date = df.iloc[next_game_index]['Date']

      # Create a dictionary containing the moving averages and next game date
      moving_averages_dict = {'Next_Game_Date': next_game_date, **moving_averages}

      # Append the dictionary to the list
      moving_averages_list.append(moving_averages_dict)

  # Convert the list of dictionaries to a DataFrame
  moving_averages_df = pd.DataFrame(moving_averages_list)

  # Now you have a DataFrame 'moving_averages_df' containing the moving averages of all statistics from game 11 to 81
  # along with the date of the next game

  moving_averages_df

  text = nhl_teams[i] #i #creating individual dataframe for each team during season 1
  moving_averages_df.insert(0,'Team name', text)
  new_df_name = nhl_teams_abbr[i]            # key for dictionary

  season1_dict[new_df_name] = moving_averages_df    # dataframe for each key



All 82 games found for Anaheim Ducks
All 82 games found for Arizona Coyotes
All 82 games found for Boston Bruins
All 82 games found for Buffalo Sabres
All 82 games found for Calgary Flames
All 82 games found for Carolina Hurricanes
All 82 games found for Chicago Blackhawks
All 82 games found for Colorado Avalanche
All 82 games found for Columbus Blue Jackets
All 82 games found for Dallas Stars
All 82 games found for Detroit Red Wings
All 82 games found for Edmonton Oilers
All 82 games found for Florida Panthers
All 82 games found for Los Angeles Kings
All 82 games found for Minnesota Wild
All 82 games found for Montreal Canadiens
All 82 games found for Nashville Predators
All 82 games found for New Jersey Devils
All 82 games found for New York Islanders
All 82 games found for New York Rangers
All 82 games found for Ottawa Senators
All 82 games found for Philadelphia Flyers
All 82 games found for Pittsburgh Penguins
All 82 games found for San Jose Sharks
All 82 games found for Seattle K

In [ ]:
season2_dict ={}

for i in range(len(nhl_teams)):
  # Replace 'TEAM_NAME' with the name of the team you want to filter
  team_name = nhl_teams[i]

  # Filter the DataFrame to get all games played by the specified team
  team_games_df = season2[season2['Team'] == team_name]

  # Verify the length of the DataFrame to ensure it contains 82 games
  if len(team_games_df) == 82:
      print("All 82 games found for", team_name,'2')
  else:
      print("The number of games found for", team_name, "is not 82:",len(team_games_df),"Games found.: i =", i)


  # If you want to reset the index of the resulting DataFrame
  team_games_df.reset_index(drop=True, inplace=True)


  team_name = nhl_teams[i] #i
  team_games_df = season2[season2['Team'] == team_name]
  # Loop through each game from game 10 to 82 (inclusive)
  # Create an empty list to store dictionaries
  moving_averages_list = []
  df = team_games_df
  df_copy = df.copy()
  df_copy.replace('-', np.nan, inplace=True)
  exclude_columns = ['TOI/GP', 'Team', 'Date']


  for game_index in range(10, 82):
      # Select data from game 1 to the current game, excluding non-numeric columns
      subset_df = df_copy.drop(columns=exclude_columns).iloc[:game_index]

      # Calculate the moving averages for all statistics
      moving_averages = subset_df.rolling(window=10, min_periods=10).mean().iloc[-1]

      # Get the date of the next game
      next_game_index = game_index   # Index of the next game
      #print("Next game index:", next_game_index)

      next_game_date = df.iloc[next_game_index]['Date']

      # Create a dictionary containing the moving averages and next game date
      moving_averages_dict = {'Next_Game_Date': next_game_date, **moving_averages}

      # Append the dictionary to the list
      moving_averages_list.append(moving_averages_dict)

  # Convert the list of dictionaries to a DataFrame
  moving_averages_df = pd.DataFrame(moving_averages_list)

  # Now you have a DataFrame 'moving_averages_df' containing the moving averages of all statistics from game 11 to 81
  # along with the date of the next game

  moving_averages_df

  text = nhl_teams[i] #i #creating individual dataframe for each team during season 1
  moving_averages_df.insert(0,'Team name', text)
  new_df_name = nhl_teams_abbr[i]            # key for dictionary

  season2_dict[new_df_name] = moving_averages_df    # dataframe for each key



All 82 games found for Anaheim Ducks 2
All 82 games found for Arizona Coyotes 2
All 82 games found for Boston Bruins 2
All 82 games found for Buffalo Sabres 2
All 82 games found for Calgary Flames 2
All 82 games found for Carolina Hurricanes 2
All 82 games found for Chicago Blackhawks 2
All 82 games found for Colorado Avalanche 2
All 82 games found for Columbus Blue Jackets 2
All 82 games found for Dallas Stars 2
All 82 games found for Detroit Red Wings 2
All 82 games found for Edmonton Oilers 2
All 82 games found for Florida Panthers 2
All 82 games found for Los Angeles Kings 2
All 82 games found for Minnesota Wild 2
All 82 games found for Montreal Canadiens 2
All 82 games found for Nashville Predators 2
All 82 games found for New Jersey Devils 2
All 82 games found for New York Islanders 2
All 82 games found for New York Rangers 2
All 82 games found for Ottawa Senators 2
All 82 games found for Philadelphia Flyers 2
All 82 games found for Pittsburgh Penguins 2
All 82 games found for Sa

In [ ]:
season3_dict ={}

for i in range(len(nhl_teams)):
  # Replace 'TEAM_NAME' with the name of the team you want to filter
  team_name = nhl_teams[i]

  # Filter the DataFrame to get all games played by the specified team
  team_games_df = season3[season3['Team'] == team_name]

  # Verify the length of the DataFrame to ensure it contains 82 games
  if len(team_games_df) == 82:
      print("All 82 games found for", team_name, '3')
  else:
      print("The number of games found for", team_name, "is not 82:",len(team_games_df),"Games found.: i =", i)


  # If you want to reset the index of the resulting DataFrame
  team_games_df.reset_index(drop=True, inplace=True)


  team_name = nhl_teams[i] #i
  team_games_df = season3[season3['Team'] == team_name]
  # Loop through each game from game 10 to 82 (inclusive)
  # Create an empty list to store dictionaries
  moving_averages_list = []
  df = team_games_df
  df_copy = df.copy()
  df_copy.replace('-', np.nan, inplace=True)
  exclude_columns = ['TOI/GP', 'Team', 'Date']


  for game_index in range(10, 82):
      # Select data from game 1 to the current game, excluding non-numeric columns
      subset_df = df_copy.drop(columns=exclude_columns).iloc[:game_index]

      # Calculate the moving averages for all statistics
      moving_averages = subset_df.rolling(window=10, min_periods=10).mean().iloc[-1]

      # Get the date of the next game
      next_game_index = game_index   # Index of the next game
      #print("Next game index:", next_game_index)

      next_game_date = df.iloc[next_game_index]['Date']

      # Create a dictionary containing the moving averages and next game date
      moving_averages_dict = {'Next_Game_Date': next_game_date, **moving_averages}

      # Append the dictionary to the list
      moving_averages_list.append(moving_averages_dict)

  # Convert the list of dictionaries to a DataFrame
  moving_averages_df = pd.DataFrame(moving_averages_list)

  # Now you have a DataFrame 'moving_averages_df' containing the moving averages of all statistics from game 11 to 81
  # along with the date of the next game

  moving_averages_df

  text = nhl_teams[i] #i #creating individual dataframe for each team during season 1
  moving_averages_df.insert(0,'Team name', text)
  new_df_name = nhl_teams_abbr[i]            # key for dictionary

  season3_dict[new_df_name] = moving_averages_df    # dataframe for each key



All 82 games found for Anaheim Ducks 3
All 82 games found for Arizona Coyotes 3
All 82 games found for Boston Bruins 3
All 82 games found for Buffalo Sabres 3
All 82 games found for Calgary Flames 3
All 82 games found for Carolina Hurricanes 3
All 82 games found for Chicago Blackhawks 3
All 82 games found for Colorado Avalanche 3
All 82 games found for Columbus Blue Jackets 3
All 82 games found for Dallas Stars 3
All 82 games found for Detroit Red Wings 3
All 82 games found for Edmonton Oilers 3
All 82 games found for Florida Panthers 3
All 82 games found for Los Angeles Kings 3
All 82 games found for Minnesota Wild 3
All 82 games found for Montreal Canadiens 3
All 82 games found for Nashville Predators 3
All 82 games found for New Jersey Devils 3
All 82 games found for New York Islanders 3
All 82 games found for New York Rangers 3
All 82 games found for Ottawa Senators 3
All 82 games found for Philadelphia Flyers 3
All 82 games found for Pittsburgh Penguins 3
All 82 games found for Sa

In [ ]:
#print(season1_dict['PIT'])
# Assuming your dictionary is called 'dataframes_dict' containing dataframes

# Initialize an empty list to store the column values
# Iterate over the key-value pairs in the dictionary
#for key, value in season1_dict.items():
    # Print the key and the type of the corresponding value
    #print(f"Key: {key}, Value Type: {type(value)}")


In [ ]:
#this is to find the earliest and latest game data of the 11th game of the season for each team

# Initialize an empty list to store the column values


column_values = []

# Iterate over the keys (dataframe names) in the dictionary
for df_name in season1_dict:
    # Access the dataframe using the key
    df = season1_dict[df_name]  # Corrected access using square brackets
    #print(df)

    # Check if the dataframe has any rows
    if not df.empty:
        # Get the value of the specific column in the first row (index 0)
        column_value = df.iloc[0]['Next_Game_Date']  # Replace 'Next_Game_Date' with the actual column name
        column_values.append(column_value)

# Now column_values contains the desired column value from the first row of each dataframe
print(column_values)

earliest_date = min(column_values)
latest_date = max(column_values)
print("Earliest date:", earliest_date)
print("Latest_date:", latest_date)#workaround so that the dictionary is populated with all the data needed to make sure following algorithms work
# not all teams play their 11th game on the same day
print(len(column_values))

['2021-11-02', '2021-11-05', '2021-11-11', '2021-11-06', '2021-11-06', '2021-11-09', '2021-11-03', '2021-11-11', '2021-11-12', '2021-11-07', '2021-11-04', '2021-11-09', '2021-11-06', '2021-11-05', '2021-11-07', '2021-11-02', '2021-11-05', '2021-11-09', '2021-11-11', '2021-11-05', '2021-11-06', '2021-11-10', '2021-11-09', '2021-11-06', '2021-11-04', '2021-11-09', '2021-11-06', '2021-11-04', '2021-11-05', '2021-11-06', '2021-11-06', '2021-11-06']
Earliest date: 2021-11-02
Latest_date: 2021-11-12
32


In [ ]:
testcase = reg1dates[reg1dates['Date'] == latest_date]
print(len(testcase))
testcase

5


,Date,Visitor,G,Home,G.1,Unnamed: 5,Att.,LOG,Notes
205,2021-11-12,Edmonton Oilers,2,Buffalo Sabres,3,NaN,8258.0,2:34,NaN
206,2021-11-12,Philadelphia Flyers,2,Carolina Hurricanes,1,NaN,18680.0,2:36,NaN
207,2021-11-12,Washington Capitals,4,Columbus Blue Jackets,3,NaN,16985.0,2:24,NaN
208,2021-11-12,Arizona Coyotes,1,Chicago Blackhawks,2,NaN,17828.0,2:32,NaN
209,2021-11-12,Calgary Flames,1,Toronto Maple Leafs,2,OT,18826.0,2:29,NaN


In [ ]:
season1_data = pd.DataFrame()
season1_data = pd.DataFrame(columns=['Next_Game_Date', 'Home_Team', 'Visit_Team', 'GP', 'W', 'L',
       'OTL', 'ROW', 'Points', 'Point %', 'CF/60', 'CA/60', 'CF%', 'FF/60',
       'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%',
       'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'SCSF/60',
       'SCSA/60', 'SCSF%', 'SCGF/60', 'SCGA/60', 'SCGF%', 'SCSH%', 'SCSV%',
       'HDCF/60', 'HDCA/60', 'HDCF%', 'HDSF/60', 'HDSA/60', 'HDSF%', 'HDGF/60',
       'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'MDCF/60', 'MDCA/60', 'MDCF%',
       'MDSF/60', 'MDSA/60', 'MDSF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'MDSH%',
       'MDSV%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDSF/60', 'LDSA/60', 'LDSF%',
       'LDGF/60', 'LDGA/60', 'LDGF%', 'LDSH%', 'LDSV%', 'SH%', 'SV%', 'PDO',
       'Home_Win'])



for p in range(len(testcase)):
  Gstats = pd.DataFrame()
  visitscore = testcase.iloc[[p],[2]].values
  homescore = testcase.iloc[[p],[4]].values

  #finding what team won

  if homescore > visitscore:
    result = 1
  else:
    result = 0

  dfv = season1_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)]] # this is to input into the dict to retrive the correct dataframe
  vstats = dfv[dfv['Next_Game_Date']== latest_date]
  dfh = season1_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)]] # this is to input into the dict to retrive the correct dataframe
  hstats = dfh[dfh['Next_Game_Date']== latest_date]

  non_numeric_columns = dfv.select_dtypes(exclude=['number']).columns


  hstats = hstats.drop(non_numeric_columns, axis = 1)

  vstats = vstats.drop(non_numeric_columns, axis = 1)

  #reset index because that matters some how
  hstats = hstats.reset_index()
  vstats = vstats.reset_index()

  Gstats = hstats-vstats
  #adding in home team and visiting team for readback purposes

  Gstats.insert(0, 'Visit_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)])

  Gstats.insert(0, 'Home_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)])
  #print(Gstats)
  # need to add back in next game data for readablitiy purposes
  #need to add home win and home lose ie 1= home wins and 0= home looses add in as last column
  Gstats.insert(0, 'Next_Game_Date', latest_date)

  Gstats['Home_Win'] = result

  season1_data = pd.concat([season1_data,Gstats], ignore_index=True)

season1_data


,Next_Game_Date,Home_Team,Visit_Team,GP,W,L,OTL,ROW,Points,Point %,...,LDGF/60,LDGA/60,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Home_Win,index
0,2021-11-12,BUF,EDM,0.0,-0.5,0.3,0.2,-0.5,-0.8,-0.40,...,0.200,-0.243,NaN,1.992,1.952,-3.790,-0.595,-0.0438,1,0.0
1,2021-11-12,CAR,PHI,0.0,0.3,-0.2,-0.1,0.3,0.5,0.25,...,0.416,-0.259,NaN,2.077,1.286,1.807,1.338,0.0316,0,0.0
2,2021-11-12,CBJ,WSH,0.0,0.2,0.1,-0.3,0.1,0.1,0.05,...,0.376,0.055,NaN,3.441,0.026,-0.070,0.424,0.0036,0,-3.0
3,2021-11-12,CHI,ARI,0.0,0.2,-0.3,0.1,0.1,0.5,0.25,...,-0.237,-0.317,NaN,-2.083,2.686,0.619,-0.948,-0.0034,1,1.0
4,2021-11-12,TOR,CGY,0.0,0.0,0.2,-0.2,0.0,-0.2,-0.10,...,-0.262,-0.003,NaN,-2.068,0.852,-2.109,-1.685,-0.0378,1,1.0


In [ ]:
season1_data = pd.DataFrame()
season1_data = pd.DataFrame(columns=['Next_Game_Date', 'Home_Team', 'Visit_Team', 'GP', 'W', 'L',
       'OTL', 'ROW', 'Points', 'Point %', 'CF/60', 'CA/60', 'CF%', 'FF/60',
       'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%',
       'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'SCSF/60',
       'SCSA/60', 'SCSF%', 'SCGF/60', 'SCGA/60', 'SCGF%', 'SCSH%', 'SCSV%',
       'HDCF/60', 'HDCA/60', 'HDCF%', 'HDSF/60', 'HDSA/60', 'HDSF%', 'HDGF/60',
       'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'MDCF/60', 'MDCA/60', 'MDCF%',
       'MDSF/60', 'MDSA/60', 'MDSF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'MDSH%',
       'MDSV%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDSF/60', 'LDSA/60', 'LDSF%',
       'LDGF/60', 'LDGA/60', 'LDGF%', 'LDSH%', 'LDSV%', 'SH%', 'SV%', 'PDO',
       'Home_Win'])
testcase

,Date,Visitor,G,Home,G.1,Unnamed: 5,Att.,LOG,Notes
205,2021-11-12,Edmonton Oilers,2,Buffalo Sabres,3,NaN,8258.0,2:34,NaN
206,2021-11-12,Philadelphia Flyers,2,Carolina Hurricanes,1,NaN,18680.0,2:36,NaN
207,2021-11-12,Washington Capitals,4,Columbus Blue Jackets,3,NaN,16985.0,2:24,NaN
208,2021-11-12,Arizona Coyotes,1,Chicago Blackhawks,2,NaN,17828.0,2:32,NaN
209,2021-11-12,Calgary Flames,1,Toronto Maple Leafs,2,OT,18826.0,2:29,NaN


In [ ]:
dates1 = reg1dates[reg1dates['Date']>= latest_date]
dates1 = dates1['Date'].unique()
print(len(dates1)) #list of dates to iterate through after latest date of 10th game played for all teams
#use this array to
print(dates1[0])
print(latest_date)

158
2021-11-12
2021-11-12


In [ ]:
season1_data = pd.DataFrame()
season1_data = pd.DataFrame(columns=['Next_Game_Date', 'Home_Team', 'Visit_Team', 'GP', 'W', 'L',
       'OTL', 'ROW', 'Points', 'Point %', 'CF/60', 'CA/60', 'CF%', 'FF/60',
       'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%',
       'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'SCSF/60',
       'SCSA/60', 'SCSF%', 'SCGF/60', 'SCGA/60', 'SCGF%', 'SCSH%', 'SCSV%',
       'HDCF/60', 'HDCA/60', 'HDCF%', 'HDSF/60', 'HDSA/60', 'HDSF%', 'HDGF/60',
       'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'MDCF/60', 'MDCA/60', 'MDCF%',
       'MDSF/60', 'MDSA/60', 'MDSF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'MDSH%',
       'MDSV%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDSF/60', 'LDSA/60', 'LDSF%',
       'LDGF/60', 'LDGA/60', 'LDGF%', 'LDSH%', 'LDSV%', 'SH%', 'SV%', 'PDO',
       'Home_Win'])

for q in range(len(dates1)):
  testcase = reg1dates[reg1dates['Date'] == dates1[q]]

  for p in range(len(testcase)):
    Gstats = pd.DataFrame()
    visitscore = testcase.iloc[[p],[2]].values
    homescore = testcase.iloc[[p],[4]].values

    #finding what team won

    if homescore > visitscore:
      result = 1
    else:
      result = 0

    dfv = season1_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)]] # this is to input into the dict to retrive the correct dataframe
    vstats = dfv[dfv['Next_Game_Date']== dates1[q]]
    dfh = season1_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)]] # this is to input into the dict to retrive the correct dataframe
    hstats = dfh[dfh['Next_Game_Date']== dates1[q]]

    non_numeric_columns = dfv.select_dtypes(exclude=['number']).columns


    hstats = hstats.drop(non_numeric_columns, axis = 1)

    vstats = vstats.drop(non_numeric_columns, axis = 1)

    #reset index because that matters some how
    hstats = hstats.reset_index()
    vstats = vstats.reset_index()

    Gstats = hstats-vstats
    #adding in home team and visiting team for readback purposes

    Gstats.insert(0, 'Visit_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)])

    Gstats.insert(0, 'Home_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)])
    #print(Gstats)
    # need to add back in next game data for readablitiy purposes
    #need to add home win and home lose ie 1= home wins and 0= home looses add in as last column
    Gstats.insert(0, 'Next_Game_Date', dates1[q])

    Gstats['Home_Win'] = result

    season1_data = pd.concat([season1_data,Gstats], ignore_index=True)


In [ ]:
season1_data = season1_data.drop(season1_data.columns[-1], axis = 1)
season1_data

,Next_Game_Date,Home_Team,Visit_Team,GP,W,L,OTL,ROW,Points,Point %,...,LDSF%,LDGF/60,LDGA/60,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Home_Win
0,2021-11-12,BUF,EDM,0.0,-0.5,0.3,0.2,-0.5,-0.8,-0.40,...,-0.534,0.200,-0.243,NaN,1.992,1.952,-3.790,-0.595,-0.0438,1
1,2021-11-12,CAR,PHI,0.0,0.3,-0.2,-0.1,0.3,0.5,0.25,...,3.987,0.416,-0.259,NaN,2.077,1.286,1.807,1.338,0.0316,0
2,2021-11-12,CBJ,WSH,0.0,0.2,0.1,-0.3,0.1,0.1,0.05,...,-2.318,0.376,0.055,NaN,3.441,0.026,-0.070,0.424,0.0036,0
3,2021-11-12,CHI,ARI,0.0,0.2,-0.3,0.1,0.1,0.5,0.25,...,-1.924,-0.237,-0.317,NaN,-2.083,2.686,0.619,-0.948,-0.0034,1
4,2021-11-12,TOR,CGY,0.0,0.0,0.2,-0.2,0.0,-0.2,-0.10,...,-7.565,-0.262,-0.003,NaN,-2.068,0.852,-2.109,-1.685,-0.0378,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,2022-04-29,SEA,SJS,0.0,0.1,0.3,-0.4,0.1,-0.2,-0.10,...,0.044,-0.340,-0.214,NaN,-1.858,-0.325,0.390,-2.493,-0.0210,1
1103,2022-04-29,STL,VGK,0.0,0.5,-0.2,-0.3,0.5,0.7,0.35,...,-9.887,-0.728,0.290,NaN,-2.299,-2.665,6.287,-1.815,0.0447,0
1104,2022-04-29,TOR,BOS,-0.1,0.1,-0.3,0.1,0.0,0.3,NaN,...,-5.435,0.055,0.000,NaN,1.103,-0.571,2.676,-2.734,-0.0004,1
1105,2022-04-29,WPG,CGY,0.0,-0.3,0.4,-0.1,-0.3,-0.7,-0.35,...,-11.759,-0.615,0.459,NaN,-3.507,-3.435,-3.419,1.291,-0.0214,1


In [ ]:
#this is to find the earliest and latest game data of the 11th game of the season for each team

# Initialize an empty list to store the column values


column_values = []

# Iterate over the keys (dataframe names) in the dictionary
for df_name in season2_dict:
    # Access the dataframe using the key
    df = season2_dict[df_name]  # Corrected access using square brackets
    #print(df)

    # Check if the dataframe has any rows
    if not df.empty:
        # Get the value of the specific column in the first row (index 0)
        column_value = df.iloc[0]['Next_Game_Date']  # Replace 'Next_Game_Date' with the actual column name
        column_values.append(column_value)

# Now column_values contains the desired column value from the first row of each dataframe
print(column_values)

earliest_date = min(column_values)
latest_date = max(column_values)
print("Earliest date:", earliest_date)
print("Latest_date:", latest_date)#workaround so that the dictionary is populated with all the data needed to make sure following algorithms work
# not all teams play their 11th game on the same day
print(len(column_values))



['2022-11-03', '2022-11-05', '2022-11-03', '2022-11-04', '2022-11-07', '2022-11-04', '2022-11-03', '2022-11-05', '2022-11-04', '2022-11-03', '2022-11-05', '2022-11-03', '2022-11-03', '2022-10-31', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-03', '2022-11-01', '2022-11-05', '2022-11-05', '2022-11-02', '2022-10-29', '2022-11-01', '2022-11-08', '2022-11-03', '2022-11-02', '2022-11-03', '2022-11-01', '2022-11-01', '2022-11-05']
Earliest date: 2022-10-29
Latest_date: 2022-11-08
32


In [ ]:
dates2 = reg2dates[reg2dates['Date']>= latest_date]
dates2 = dates2['Date'].unique()
print(len(dates2)) #list of dates to iterate through after latest date of 10th game played for all teams
#use this array to
print(dates2[0])
print(latest_date)



149
2022-11-08
2022-11-08


In [ ]:
season2_data = pd.DataFrame()
season2_data = pd.DataFrame(columns=['Next_Game_Date', 'Home_Team', 'Visit_Team', 'GP', 'W', 'L',
       'OTL', 'ROW', 'Points', 'Point %', 'CF/60', 'CA/60', 'CF%', 'FF/60',
       'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%',
       'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'SCSF/60',
       'SCSA/60', 'SCSF%', 'SCGF/60', 'SCGA/60', 'SCGF%', 'SCSH%', 'SCSV%',
       'HDCF/60', 'HDCA/60', 'HDCF%', 'HDSF/60', 'HDSA/60', 'HDSF%', 'HDGF/60',
       'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'MDCF/60', 'MDCA/60', 'MDCF%',
       'MDSF/60', 'MDSA/60', 'MDSF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'MDSH%',
       'MDSV%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDSF/60', 'LDSA/60', 'LDSF%',
       'LDGF/60', 'LDGA/60', 'LDGF%', 'LDSH%', 'LDSV%', 'SH%', 'SV%', 'PDO',
       'Home_Win'])

for q in range(len(dates2)):
  testcase = reg2dates[reg2dates['Date'] == dates2[q]]

  for p in range(len(testcase)):
    Gstats = pd.DataFrame()
    visitscore = testcase.iloc[[p],[2]].values
    homescore = testcase.iloc[[p],[4]].values

    #finding what team won

    if homescore > visitscore:
      result = 1
    else:
      result = 0

    dfv = season2_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)]] # this is to input into the dict to retrive the correct dataframe
    vstats = dfv[dfv['Next_Game_Date']== dates2[q]]
    dfh = season2_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)]] # this is to input into the dict to retrive the correct dataframe
    hstats = dfh[dfh['Next_Game_Date']== dates2[q]]

    non_numeric_columns = dfv.select_dtypes(exclude=['number']).columns


    hstats = hstats.drop(non_numeric_columns, axis = 1)

    vstats = vstats.drop(non_numeric_columns, axis = 1)

    #reset index because that matters some how
    hstats = hstats.reset_index()
    vstats = vstats.reset_index()

    Gstats = hstats-vstats
    #adding in home team and visiting team for readback purposes

    Gstats.insert(0, 'Visit_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)])

    Gstats.insert(0, 'Home_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)])
    #print(Gstats)
    # need to add back in next game data for readablitiy purposes
    #need to add home win and home lose ie 1= home wins and 0= home looses add in as last column
    Gstats.insert(0, 'Next_Game_Date', dates2[q])

    Gstats['Home_Win'] = result

    season2_data = pd.concat([season2_data,Gstats], ignore_index=True)


In [ ]:
season2_data = season2_data.drop(season2_data.columns[-1], axis = 1)
season2_data

,Next_Game_Date,Home_Team,Visit_Team,GP,W,L,OTL,ROW,Points,Point %,...,LDSF%,LDGF/60,LDGA/60,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Home_Win
0,2022-11-08,BUF,ARI,0.0,0.2,-0.1,-0.1,0.2,0.3,0.15,...,8.249,0.053,0.361,NaN,-0.413,-2.843,0.713,0.703,0.0140,0
1,2022-11-08,DET,MTL,0.0,0.1,-0.2,0.1,0.1,0.3,0.15,...,-13.607,-0.478,-0.019,NaN,-2.432,1.633,-1.882,2.900,0.0102,0
2,2022-11-08,LAK,MIN,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,10.770,-0.005,0.014,NaN,0.055,-2.198,1.128,-2.127,-0.0099,1
3,2022-11-08,NJD,CGY,0.0,0.5,-0.3,-0.2,0.5,0.8,0.40,...,-6.136,-0.029,0.131,NaN,-1.197,4.583,0.763,1.449,0.0220,1
4,2022-11-08,NYR,NYI,0.0,-0.2,-0.1,0.3,-0.2,-0.1,-0.05,...,3.707,0.125,-0.050,NaN,1.231,-0.738,-2.681,-2.182,-0.0485,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,2023-04-13,SEA,VGK,0.0,0.1,0.2,-0.3,0.2,-0.1,-0.05,...,9.299,0.311,0.266,NaN,2.185,-3.131,1.408,-5.110,-0.0369,0
1113,2023-04-13,TBL,DET,0.0,-0.1,0.2,-0.1,-0.1,-0.3,-0.15,...,7.396,-0.105,0.020,NaN,-1.769,-0.472,-4.047,2.791,-0.0126,1
1114,2023-04-13,WSH,NJD,0.0,-0.4,0.2,0.2,-0.4,-0.6,-0.30,...,-9.672,-0.249,0.345,NaN,-1.349,-3.357,-0.405,-0.220,-0.0062,0
1115,2023-04-14,CBJ,BUF,0.0,-0.5,0.4,0.1,-0.3,-0.9,-0.45,...,-7.925,0.115,-0.106,NaN,0.498,1.683,-2.598,-4.420,-0.0701,0


In [ ]:
#this is to find the earliest and latest game data of the 11th game of the season for each team

# Initialize an empty list to store the column values


column_values = []

# Iterate over the keys (dataframe names) in the dictionary
for df_name in season3_dict:
    # Access the dataframe using the key
    df = season3_dict[df_name]  # Corrected access using square brackets
    #print(df)

    # Check if the dataframe has any rows
    if not df.empty:
        # Get the value of the specific column in the first row (index 0)
        column_value = df.iloc[0]['Next_Game_Date']  # Replace 'Next_Game_Date' with the actual column name
        column_values.append(column_value)

# Now column_values contains the desired column value from the first row of each dataframe
print(column_values)

earliest_date = min(column_values)
latest_date = max(column_values)
print("Earliest date:", earliest_date)
print("Latest_date:", latest_date)#workaround so that the dictionary is populated with all the data needed to make sure following algorithms work
# not all teams play their 11th game on the same day
print(len(column_values))



['2023-11-05', '2023-11-04', '2023-11-04', '2023-11-03', '2023-11-04', '2023-11-02', '2023-11-05', '2023-11-07', '2023-11-04', '2023-11-06', '2023-11-02', '2023-11-06', '2023-11-06', '2023-11-04', '2023-11-04', '2023-11-04', '2023-11-04', '2023-11-05', '2023-11-07', '2023-11-04', '2023-11-08', '2023-11-03', '2023-11-07', '2023-11-04', '2023-11-02', '2023-11-07', '2023-11-04', '2023-11-04', '2023-11-04', '2023-11-02', '2023-11-08', '2023-11-04']
Earliest date: 2023-11-02
Latest_date: 2023-11-08
32


In [ ]:
dates3 = reg3dates[reg3dates['Date']>= latest_date]
dates3 = dates3['Date'].unique()
print(len(dates3)) #list of dates to iterate through after latest date of 10th game played for all teams
#use this array to
print(dates3[0])
print(latest_date)


154
2023-11-08
2023-11-08


In [ ]:
season3_data = pd.DataFrame()
season3_data = pd.DataFrame(columns=['Next_Game_Date', 'Home_Team', 'Visit_Team', 'GP', 'W', 'L',
       'OTL', 'ROW', 'Points', 'Point %', 'CF/60', 'CA/60', 'CF%', 'FF/60',
       'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%',
       'xGF/60', 'xGA/60', 'xGF%', 'SCF/60', 'SCA/60', 'SCF%', 'SCSF/60',
       'SCSA/60', 'SCSF%', 'SCGF/60', 'SCGA/60', 'SCGF%', 'SCSH%', 'SCSV%',
       'HDCF/60', 'HDCA/60', 'HDCF%', 'HDSF/60', 'HDSA/60', 'HDSF%', 'HDGF/60',
       'HDGA/60', 'HDGF%', 'HDSH%', 'HDSV%', 'MDCF/60', 'MDCA/60', 'MDCF%',
       'MDSF/60', 'MDSA/60', 'MDSF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'MDSH%',
       'MDSV%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDSF/60', 'LDSA/60', 'LDSF%',
       'LDGF/60', 'LDGA/60', 'LDGF%', 'LDSH%', 'LDSV%', 'SH%', 'SV%', 'PDO',
       'Home_Win'])

for q in range(len(dates3)):
  testcase = reg3dates[reg3dates['Date'] == dates3[q]]

  for p in range(len(testcase)):
    Gstats = pd.DataFrame()
    visitscore = testcase.iloc[[p],[2]].values
    homescore = testcase.iloc[[p],[4]].values

    #finding what team won

    if homescore > visitscore:
      result = 1
    else:
      result = 0

    dfv = season3_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)]] # this is to input into the dict to retrive the correct dataframe
    vstats = dfv[dfv['Next_Game_Date']== dates3[q]]
    dfh = season3_dict[nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)]] # this is to input into the dict to retrive the correct dataframe
    hstats = dfh[dfh['Next_Game_Date']== dates3[q]]

    non_numeric_columns = dfv.select_dtypes(exclude=['number']).columns


    hstats = hstats.drop(non_numeric_columns, axis = 1)

    vstats = vstats.drop(non_numeric_columns, axis = 1)

    #reset index because that matters some how
    hstats = hstats.reset_index()
    vstats = vstats.reset_index()

    Gstats = hstats-vstats
    #adding in home team and visiting team for readback purposes

    Gstats.insert(0, 'Visit_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[1]].values)])

    Gstats.insert(0, 'Home_Team',nhl_teams_abbr[nhl_teams.index(testcase.iloc[[p],[3]].values)])
    #print(Gstats)
    # need to add back in next game data for readablitiy purposes
    #need to add home win and home lose ie 1= home wins and 0= home looses add in as last column
    Gstats.insert(0, 'Next_Game_Date', dates3[q])

    Gstats['Home_Win'] = result

    season3_data = pd.concat([season3_data,Gstats], ignore_index=True)

In [ ]:
season3_data = season3_data.drop(season3_data.columns[-1], axis = 1)
season3_data


,Next_Game_Date,Home_Team,Visit_Team,GP,W,L,OTL,ROW,Points,Point %,...,LDSF%,LDGF/60,LDGA/60,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Home_Win
0,2023-11-08,TOR,OTT,0.0,0.0,-0.2,0.2,0.0,0.2,0.10,...,5.244,-0.274,-0.283,NaN,-1.923,1.468,-1.646,1.756,0.0010,0
1,2023-11-08,VGK,LAK,0.0,0.1,0.0,-0.1,-0.2,0.1,0.05,...,-16.978,-0.298,0.202,NaN,-1.509,-0.436,-1.537,2.079,0.0053,0
2,2023-11-08,WSH,FLA,0.0,-0.1,0.1,0.0,-0.3,-0.2,-0.10,...,-8.283,-0.530,0.016,NaN,-3.174,1.672,-3.147,-2.342,-0.0550,0
3,2023-11-09,BOS,NYI,0.0,0.4,-0.2,-0.2,0.3,0.6,0.30,...,-2.406,-0.141,-0.624,NaN,-0.293,3.551,3.704,0.288,0.0400,1
4,2023-11-09,CBJ,DAL,0.0,-0.3,0.1,0.2,-0.3,-0.4,-0.20,...,6.067,0.282,0.090,NaN,1.127,0.545,-2.165,-1.844,-0.0401,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,2024-04-18,COL,EDM,0.0,-0.2,0.2,0.0,-0.2,-0.4,-0.20,...,-5.303,-0.464,0.280,NaN,-2.946,-1.661,-1.893,-2.223,-0.0411,1
1119,2024-04-18,LAK,CHI,0.0,0.2,-0.2,0.0,0.2,0.4,0.20,...,15.129,0.217,-0.220,NaN,1.556,1.130,0.905,-1.618,-0.0068,1
1120,2024-04-18,MIN,SEA,0.0,0.1,-0.2,0.1,0.1,0.3,0.15,...,-3.642,0.130,0.007,NaN,1.063,1.253,-0.434,0.733,0.0030,0
1121,2024-04-18,VGK,ANA,0.0,0.4,-0.4,0.0,0.4,0.8,0.40,...,0.736,-0.360,1.174,NaN,-3.661,-9.402,3.788,-0.720,0.0309,0


# ***Testing***

Split Data

In [ ]:
train = pd.concat([season1_data, season2_data, season3_data], ignore_index=True)
#print(train.iloc[2].values)
columns_with_nan = train.columns[train.isna().any()].tolist()
columns_with_nan
train = train.drop(columns_with_nan, axis=1)
train
print(type(result))

<class 'int'>


In [ ]:
X = train.loc[:,'W':'PDO'].values

Yint = train.iloc[:, -1].values
y = Yint.astype(np.int64)
uniqueVal = np.unique(y)
print(uniqueVal)
print(type(y[0]))
# Split these data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=123)

[0 1]
<class 'numpy.int64'>


Logsitic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_2 = LogisticRegression(max_iter= 5000).fit(X_train,y_train)

In [ ]:
print("Model Coefficients: ", log_reg_2.coef_)


Model Coefficients:  [[-2.35782687e-02 -4.14673673e-01  1.78351384e-01  6.03797163e-01
   1.31194846e-01 -1.66189283e-02  3.49362032e-02  3.22285093e-02
  -1.79504796e-01  1.97060758e-01  3.43738319e-01  1.46897265e-03
  -2.04041619e-01 -1.47232407e-01  3.99571157e-01  5.00226379e-01
   5.38783255e-02  1.94631068e-01  3.10361143e-02 -1.23525371e-01
  -1.37177895e-01  1.32985038e-01  8.17762060e-02 -2.25781363e-02
  -4.99619565e-01 -1.19820337e-01  3.93878247e-02  1.06663665e-02
  -3.22393706e-02 -7.08280823e-02  2.68104759e-02  1.12343757e-02
   1.74140244e-01  5.09275296e-02 -1.97133724e-01  6.11775294e-03
   5.71870460e-02 -4.54928918e-03  3.53159504e-02  8.23964896e-02
  -1.30346605e-01 -1.33652842e-02 -3.14732328e-01 -1.67369032e-01
  -7.60229471e-03 -3.42681280e-02  8.59978629e-03  1.71841256e-01
   6.63571153e-03 -2.40449169e-02 -4.29161036e-01 -3.78849643e-01
  -1.20924206e-02  5.02927799e-02  3.60588125e-04]]


In [ ]:
print("\nModel Intercept: ",log_reg_2.intercept_)


Model Intercept:  [0.16106096]


In [ ]:
# Predict responses
pred_2 = log_reg_2.predict(X_test)
prediction_2 = list(map(round, pred_2))

from sklearn.metrics import accuracy_score

# Accuracy score
print('\nTest accuracy = ', accuracy_score(y_test, prediction_2))


Test accuracy =  0.5830346475507766


In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion matrix
cm = confusion_matrix(y_test, prediction_2)
print ("\nConfusion Matrix : \n", cm)


Confusion Matrix : 
 [[212 196]
 [153 276]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB



# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)

# Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

# Support Vector Machine (SVM) Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

# K-Nearest Neighbors (KNN) Classifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Accuracy:", knn_accuracy)


# Gaussian Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

Decision Tree Accuracy: 0.5125448028673835
Random Forest Accuracy: 0.5579450418160096
SVM Accuracy: 0.5651135005973715
KNN Accuracy: 0.5686977299880526
Naive Bayes Accuracy: 0.5794504181600956


In [ ]:
import xgboost as xgb

# Gradient Boosting Machine (GBM) Classifier
gbm_classifier = xgb.XGBClassifier()
gbm_classifier.fit(X_train, y_train)
gbm_predictions = gbm_classifier.predict(X_test)
gbm_accuracy = accuracy_score(y_test, gbm_predictions)
print("GBM Accuracy:", gbm_accuracy)


GBM Accuracy: 0.5507765830346476


In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


# Scale features (important for neural networks)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 classes for iris dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Neural Network Accuracy:", test_accuracy)


21/21 [==============================] - 0s 2ms/step - loss: 1.1037 - accuracy: 0.5343
Neural Network Accuracy: 0.5343283414840698
